# 8. Factory.FromAsync

(简APM模式(委托)转换为任务)(BeginXXX和EndXXX)

In [1]:
using System.Threading;
using System.Threading.Tasks;

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
public static class TaskClass
{
    public delegate string AsynchronousTask(string threadName);

    public static string TestMethod(string threadName)
    {
        Console.WriteLine("Starting... ...");
        Console.WriteLine("Is thread pool thread: {0}", Thread.CurrentThread.IsThreadPoolThread);
        Thread.Sleep(TimeSpan.FromSeconds(2));
        Thread.CurrentThread.Name = threadName;

        return string.Format("Thread name: {0}", Thread.CurrentThread.Name);
    }

    public static void Callback(IAsyncResult ar)
    {
        Console.WriteLine("Starting a callback... ...");
        Console.WriteLine("State passed to a callbak: {0}", ar.AsyncState);
        Console.WriteLine("Is thread pool thread: {0}", Thread.CurrentThread.IsThreadPoolThread);
        Console.WriteLine("Thread pool worker thread id: {0}", Thread.CurrentThread.ManagedThreadId);
    }
}

## 8.1 带回调方式的

In [3]:
//执行的流程：Test——>Callback——>task.ContinueWith
TaskClass.AsynchronousTask d = TaskClass.TestMethod;
Console.WriteLine("Option 1");
Task<string> task = Task<string>.Factory.FromAsync(d.BeginInvoke("AsyncTaskThread", TaskClass.Callback, "a delegate asynchronous call"), d.EndInvoke);
task.ContinueWith(t => Console.WriteLine("Callback is finished, now running a continuation! Result: {0}", t.Result));

while (!task.IsCompleted)
{
    Console.WriteLine(task.Status);
    Thread.Sleep(TimeSpan.FromSeconds(0.5));
}
Console.WriteLine(task.Status);

Option 1



(5,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



Unhandled exception: System.PlatformNotSupportedException: Operation is not supported on this platform.
   at Submission#5.TaskClass.AsynchronousTask.BeginInvoke(String threadName, AsyncCallback callback, Object object)
   at Submission#6.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## 8.2 不带回调方式的

In [4]:
//执行的流程：Test--->task.ContinueWith
TaskClass.AsynchronousTask d = TaskClass.TestMethod;
Task<string> task = Task<string>.Factory.FromAsync(d.BeginInvoke, d.EndInvoke, "AsyncTaskThread", "a delegate asynchronous call");
task.ContinueWith(t => Console.WriteLine("Task is completed, now running a continuation! Result: {0}", t.Result));

while (!task.IsCompleted)
{
    Console.WriteLine(task.Status);
    Thread.Sleep(TimeSpan.FromSeconds(0.5));
}
Console.WriteLine(task.Status);


(4,1): warning CS4014: Because this call is not awaited, execution of the current method continues before the call is completed. Consider applying the 'await' operator to the result of the call.



Unhandled exception: System.PlatformNotSupportedException: Operation is not supported on this platform.
   at Submission#5.TaskClass.AsynchronousTask.BeginInvoke(String threadName, AsyncCallback callback, Object object)
   at System.Threading.Tasks.TaskFactory`1.FromAsyncImpl[TArg1](Func`4 beginMethod, Func`2 endFunction, Action`1 endAction, TArg1 arg1, Object state, TaskCreationOptions creationOptions)
   at System.Threading.Tasks.TaskFactory`1.FromAsync[TArg1](Func`4 beginMethod, Func`2 endMethod, TArg1 arg1, Object state)
   at Submission#7.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

## 8.3 另外几个示例

Task启动带参数和返回值的函数任务

```C#
//Task启动带参数和返回值的函数任务
//下面的例子test2 是个带参数和返回值的函数。
private int test2(object i)
{
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = true;
    }));
    System.Threading.Thread.Sleep(3000);
    MessageBox.Show("hello:" + i);
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = false;
    }));
    return 0;
}

//测试调用
private void call()
{
    //Func<string, string> funcOne = delegate(string s){ return "fff"; };
    object i = 55;
    var t = Task<int>.Factory.StartNew(new Func<object, int>(test2), i);
}
```

```C#
//= 下载网站源文件例子 == == == == == == == == == == == ==
//HttpClient 引用System.Net.Http
private async Task< int> test2(object i)
{
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = true;
    }));

    HttpClient client = new HttpClient();
    var a = await client.GetAsync("http://www.baidu.com");
    Task<string> s = a.Content.ReadAsStringAsync();
    MessageBox.Show (s.Result);

    //System.Threading.Thread.Sleep(3000);
    //MessageBox.Show("hello:"+ i);
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = false;
    }));
    return 0;
}

async private void call()
{
    //Func<string, string> funcOne = delegate(string s){ return "fff"; };
    object i = 55;
    var t = Task<Task<int>>.Factory.StartNew(new Func<object, Task<int>>(test2), i);
}
```

```C#
//----------或者----------
private async void test2()
{
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = true;
    }));
    HttpClient client = new HttpClient();
    var a = await client.GetAsync("http://www.baidu.com");
    Task<string> s = a.Content.ReadAsStringAsync();
    MessageBox.Show (s.Result);
    this.Invoke(new Action(() =>
    {
        pictureBox1.Visible = false;
    }));
}

private void call()
{
    var t = Task.Run(new Action(test2));
    //相当于
    //Thread th= new Thread(new ThreadStart(test2));
    //th.Start();
}
```